# **Flask Site Test**

**Note: Must use GPU**

To run the flask site, download templates.zip and ingredient_classifier.pkl from the website_prototype folder in the team-apatosaurus github repository. Disregard option 1 and follow steps in option 2 to upload and move them appropriately

In [1]:
!pip install flask-ngrok

**Option 1: Copy HTML templates and pickled model from Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#copy templates & .pkl from drive to VM
%cp -av "/content/drive/MyDrive/flask_test/templates" "/content/"
%cp -av "/content/drive/MyDrive/flask_test/ingredient_classifier.pkl" "/content/"

'/content/drive/MyDrive/flask_test/templates' -> '/content/templates'
'/content/drive/MyDrive/flask_test/templates/uploadImage.html' -> '/content/templates/uploadImage.html'
'/content/drive/MyDrive/flask_test/templates/output_prediction.html' -> '/content/templates/output_prediction.html'
'/content/drive/MyDrive/flask_test/ingredient_classifier.pkl' -> '/content/ingredient_classifier.pkl'


**Option 2: Upload HTML templates and pickled model from local machine**

First, upload the templates.zip and ingredient_classifier.pkl to colab's files. Then, run the cell below

In [3]:
!unzip "templates.zip" -d "/content/"
#!mv "/templates" "/content/"
!mv "ingredient_classifier.pkl" "/content/"

Archive:  templates.zip
replace /content/templates/output_prediction.html? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
mv: 'ingredient_classifier.pkl' and '/content/ingredient_classifier.pkl' are the same file


**Define Dictionary Mapping Labels to Ingredient Names**

In [4]:
#define dictionary of labels to ingredient names

ingredient_dict = {
    0: "apple",
    1: "avocado",
    2: "banana",
    3: "beef",
    4: "bellpeppers",
    5: "bread",
    6: "broccoli",
    7: "cabbage",
    8: "cheese",
    9: "chicken",
    10: "corn",
    11: "cucumber",
    12: "egg",
    13: "eggplant",
    14: "greenbeans",
    15: "lemon",
    16: "lettuce",
    17: "mushroom",
    18: "olives",
    19: "onions",
    20: "pasta",
    21: "potatoes",
    22: "rice",
    23: "salmon",
    24: "spinach",
    25: "tomato",

}

**Flask Website Prototype**

This website allows user to upload an image and will output the trained model's prediction of what ingredient is in the image

In [5]:
from flask_ngrok import run_with_ngrok

from flask import Flask, render_template, request, redirect, send_from_directory
from werkzeug.utils import secure_filename
import os
import pickle
import glob
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms

app = Flask(__name__)
run_with_ngrok(app)

infile = open('ingredient_classifier.pkl', 'rb')
model = pickle.load(infile)
infile.close()

app.config["image_uploads"] = 'image-uploads/sample'

@app.route('/', methods=['GET','POST'])
def upload_image():
    files = glob.glob('./image-uploads/sample/*')
    for f in files:
        os.remove(f)

    os.makedirs("./image-uploads/sample", exist_ok=True)
    if request.method == "POST":
        #if request.files:
            image = request.files["image"]
            filename = secure_filename(image.filename)
            path = os.path.join(app.config["image_uploads"], filename)
            image.save(path)
            xform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
            inputs = datasets.ImageFolder('./image-uploads', transform=xform)
            loader = torch.utils.data.DataLoader(inputs, batch_size = 4, shuffle=True)
            model.eval()
            with torch.no_grad():
              for samples,_ in loader:
                samples = samples.to(device)
                outs = model(samples)
                _, preds = torch.max(outs.detach(), 1)
            
            preds_list = preds.tolist()
            #for p in preds_list:
            #  print(p)
            print(ingredient_dict[preds_list[0]])
            print("finished")
            return render_template("output_prediction.html", ingredient=ingredient_dict[preds_list[0]])

    return render_template("uploadImage.html")

@app.route('/output_prediction/<i>')
def output_prediction(i):
  return render_template("output_prediction.html", ingredient=i)

In [6]:
torch.cuda.device_count()
device = torch.device('cuda:0')

In [7]:
#run website
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://092e4df9e85f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/May/2021 09:04:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2021 09:04:17] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/May/2021 09:05:14] "POST / HTTP/1.1" 200 -


bread
finished


127.0.0.1 - - [08/May/2021 09:05:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2021 09:05:39] "POST / HTTP/1.1" 200 -


onions
finished
